# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**:

**Professor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Vicente-Siloe-Lopez-Rodriguez") \
    .master("spark://96ab4d7574c9:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 14:03:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 14:03:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
from pyspark.sql import SparkSession

# Iniciar la sesión de Spark
spark = SparkSession.builder.appName("Big Data Exam").getOrCreate()

# Cargar los archivos CSV de empleados y departamentos
employees_df = spark.read.csv('/exams/midterm/employees.csv', header=True, inferSchema=True)
departments_df = spark.read.csv('/exams/midterm/departments.csv', header=True, inferSchema=True)

# Mostrar las primeras filas de ambos DataFrames
employees_df.show(5)
departments_df.show(5)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/exams/midterm/employees.csv.

In [5]:
from pyspark.sql.functions import col, from_json

# Suponiendo que el campo employee_info contiene un JSON, extraemos las columnas
employee_info_schema = "name STRING, department_id INT, salary DOUBLE, hire_date DATE"
employees_df = employees_df.withColumn("employee_info", from_json(col("employee_info"), employee_info_schema))

# Extraer las columnas deseadas
employees_info_df = employees_df.select(
    col("employee_info.name").alias("name"),
    col("employee_info.department_id").alias("department_id"),
    col("employee_info.salary").alias("salary"),
    col("employee_info.hire_date").alias("hire_date")
)

employees_info_df.show(5)

NameError: name 'employees_df' is not defined

In [ ]:
# Realizar la unión entre empleados y departamentos
joined_df = employees_info_df.join(departments_df, employees_info_df.department_id == departments_df.department_id)

joined_df.show(5)

In [ ]:
from pyspark.sql.functions import when

# Agregar una nueva columna 'salary_category' usando la función when()
employees_with_salary_category = joined_df.withColumn(
    "salary_category", 
    when(col("salary") >= 55000, "High").otherwise("Low")
)

employees_with_salary_category.show(5)

In [ ]:
# Filtrar empleados con salarios "High" y "Low"
high_salary_df = employees_with_salary_category.filter(col("salary_category") == "High")
low_salary_df = employees_with_salary_category.filter(col("salary_category") == "Low")

# Calcular el salario promedio por departamento para los dos grupos
high_salary_avg = high_salary_df.groupBy("department_name").avg("salary").withColumnRenamed("avg(salary)", "avg_salary")
low_salary_avg = low_salary_df.groupBy("department_name").avg("salary").withColumnRenamed("avg(salary)", "avg_salary")

high_salary_avg.show(5)
low_salary_avg.show(5)

In [ ]:
# Top 5 empleados con salarios más altos en la categoría "High"
top_5_high_salary = high_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_high_salary.show(5)

# Top 5 empleados con salarios más altos en la categoría "Low"
top_5_low_salary = low_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_low_salary.show(5)

In [ ]:
from pyspark.sql.functions import datediff, current_date

# Calcular los años en la empresa
employees_with_years = employees_with_salary_category.withColumn(
    "years_in_company", 
    (datediff(current_date(), col("hire_date")) / 365).cast("int")
)

# Encontrar el empleado con más años en la empresa
most_experienced = employees_with_years.orderBy(col("years_in_company").desc()).limit(1)
most_experienced.show(1)

# Encontrar la lista de empleados con más de un cierto número de años en la empresa
employees_more_than_5_years = employees_with_years.filter(col("years_in_company") > 5)
employees_more_than_5_years.show(5)